In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
import numpy as np
import tensorflow as tf


In [2]:
train_df = pd.read_excel('https://github.com/pykwon/python/blob/master/testdata_utf8/hd_carprice.xlsx?raw=True', sheet_name='train')
test_df = pd.read_excel('https://github.com/pykwon/python/blob/master/testdata_utf8/hd_carprice.xlsx?raw=True', sheet_name='test')
print(train_df.head(3))
print(test_df.head(3))

x_train = train_df.drop(['가격'], axis=1) # feature
x_test = test_df.drop(['가격'], axis=1)

y_train = train_df[['가격']] # label
y_test= test_df[['가격']]

print(x_train.head(2))
print(x_train.columns)
print(x_train.shape)

print(set(x_train.종류)) # {'대형', '중형', '준중형', '소형'}
print(set(x_train.연료)) # {'LPG', '디젤', '가솔린'}
print(set(x_train.변속기)) # {'수동', '자동'}

# 종류, 연료, 변속기 열에 대해서는 labelEncoder(), OneHotEncoder()를 등을 적용
transformer = make_column_transformer((OneHotEncoder(), ['종류','연료','변속기']),
                                      remainder='passthrough')
# remainder='passthrough' : default는 drop. 열이 transformer에 전달
transformer.fit(x_train)
x_train = transformer.transform(x_train) # 3개의 칼럼을 포함해 모든 칼럼이 표준화됨
x_test = transformer.transform(x_test)
print(x_train[:2], x_train.shape) # (71, 16)
print(y_train.shape) # (71, 1)

     가격    년식   종류    연비   마력    토크   연료  하이브리드   배기량    중량 변속기
0  1885  2015  준중형  11.8  172  21.0  가솔린      0  1999  1300  자동
1  2190  2015  준중형  12.3  204  27.0  가솔린      0  1591  1300  자동
2  1135  2015   소형  15.0  100  13.6  가솔린      0  1368  1035  수동
     가격    년식  종류    연비   마력    토크   연료  하이브리드   배기량    중량 변속기
0  1915  2015  대형   6.8  159  23.0  LPG      0  2359  1935  수동
1  1164  2012  소형  13.3  108  13.9  가솔린      0  1396  1035  자동
2  2817  2015  중형  14.4  184  41.0   디젤      0  1995  1792  자동
     년식   종류    연비   마력    토크   연료  하이브리드   배기량    중량 변속기
0  2015  준중형  11.8  172  21.0  가솔린      0  1999  1300  자동
1  2015  준중형  12.3  204  27.0  가솔린      0  1591  1300  자동
Index(['년식', '종류', '연비', '마력', '토크', '연료', '하이브리드', '배기량', '중량', '변속기'], dtype='object')
(71, 10)
{'대형', '준중형', '소형', '중형'}
{'LPG', '디젤', '가솔린'}
{'자동', '수동'}
[[0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 1.000e+00 2.015e+03 1.180e+01 1.720e+02 2.100e+01 0.000e+00
  1.999e+03 1.30

In [28]:
# Function API로 모델 작성
input = tf.keras.layers.Input(shape=(16,))
net = tf.keras.layers.Dense(32, activation='relu')(input)
net = tf.keras.layers.Dense(32, activation='relu')(net)
net = tf.keras.layers.Dense(1, activation='linear')(net)

model = tf.keras.models.Model(input, net)

model.compile(optimizer='adam', loss='mse', metrics=['mse'])
print(model.summary())

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.fit(x_train, y_train, epochs=2000, validation_data=(x_test, y_test), verbose=2, callbacks=[early_stop])

print('evaluate : ', model.evaluate(x_test, y_test))



Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)     │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,633 (6.38 KB)

 Trainable params: 1,633 (6.38 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/2000
3/3 - 1s - 314ms/step - loss: 3615428.5000 - mse: 3615428.5000 - val_loss: 7627152.0000 - val_mse: 7627152.0000
Epoch 2/2000
3/3 - 0s - 21ms/step - loss: 3054116.0000 - mse: 3054116.0000 - val_loss: 7038985.0000 - val_mse: 7038985.0000
Epoch 3/2000
3/3 - 0s - 20ms/step - loss: 2596584.5000 - mse: 2596584.5000 - val_loss: 6562015.0000 - val_mse: 6562015.0000
Epoch 4/2000
3/3 - 0s - 19ms/step - loss: 2288490.2500 - mse: 2288490.2500 - val_loss: 6162730.0000 - val_mse: 6162730.0000
Epoch 5/2000
3/3 - 0s - 20ms/step - loss: 2015516.0000 - mse: 2015516.0000 - val_loss: 5866715.0000 - val_mse: 5866715.0000
Epoch 6/2000
3/3 - 0s - 27ms/step - loss: 1864044.2500 - mse: 1864044.2500 - val_loss: 5639743.0000 - val_mse: 5639743.0000
Epoch 7/2000
3/3 - 0s - 23ms/step - loss: 1783595.2500 - mse: 1783595.2500 - val_loss: 5480179.0000 - val_mse: 5480179.0000
Epoch 8/2000
3/3 - 0s - 20ms/step - loss: 1782067.7500 - mse: 1782067.7500 - val_loss: 5357755.0000 - val_mse: 5357755.0000
Ep

In [30]:
y_predict = model.predict(x_test)
print('예측값 : ', y_predict[:5].ravel())
print('실제값 : ', y_test[:5].values.ravel())

# 새로운 데이터 입력
new_x_test = [['2015', '대형',11.8,172,21.0,'가솔린',0,1999,1300,'자동']]
new_x_test = pd.DataFrame(new_x_test, columns=['년식', '종류', '연비', '마력', '토크', '연료', '하이브리드', '배기량', '중량', '변속기'])

print(new_x_test)

new_x_test = transformer.transform(new_x_test)
# new_predict = model.predict(new_x_test)
# print('예측값 : ', new_predict.ravel())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
예측값 :  [2624.8276 1264.2494 2030.791  2734.5176 2629.2764]
실제값 :  [1915 1164 2817 2160 1915]
     년식  종류    연비   마력    토크   연료  하이브리드   배기량    중량 변속기
0  2015  대형  11.8  172  21.0  가솔린      0  1999  1300  자동
